### 快速排序的过程
1. 分解:如果S有至少2个元素(如果S只有1个或0个元素,什么都不用做),从S中选择一个特定的元素x,称之为基准值(pivot).
一般情况下,选择S中最后一个元素作为基准值.从S中移除所有的元素,并把它们放到3个序列中:
* L储存S中小于x的元素
* E储存S中等于x的元素
* G储存S中大于x的元素

如果S中元素是相异的,那么E将只含有一个元素---基准值自己

2. 解决子问题:递归地排序序列L和G
3. 合并:把S中的元素按照先插入L中的元素,然后插入E中的元素,最后插入G中的元素的顺序放回

时间复杂度(主定理分析易知):
* 最坏时间复杂度:$O(n^2)$
* 最好时间复杂度:$O(n \log n)$,发生在序列由不同的元素构成,且子序列L与G的大小大致相等的情况

In [1]:
class Empty(Exception):
    """Error attempting to access an element from an empty container"""
    pass

class LinkedQueue:
    class _Node:
        def __init__(self, element, pointer):  # initialize node's fields
            self._element = element  # reference to user's element
            self._pointer = pointer  # reference th next node

    def __init__(self):
        """Create an empty queue"""
        self._head = None
        self._tail = None
        self._size = 0 # number of queue elements

    def __len__(self):
        """Return the number of elements in the queue"""
        return self._size

    def is_empty(self):
        """Return True if the queue is empty"""
        return self._size == 0

    def first(self):
        """Return (but do not remove) the element at the front of the queue """
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._head._element

    def dequeue(self):
        """Remove and return the first element of the queue"""
        if self.is_empty():
            raise  Empty('Queue is empty')
        answer = self._head._element
        node = self._head
        self._head = self._head._pointer
        node._pointer = node._element = None # 显式的删除该节点
        self._size -= 1
        if self.is_empty(): # special case as queue is empty
            self._tail = None # removed head had been the tail
        return answer

    # 在链表尾节点后添加节点,尾节点为最后进入的元素
    def enqueue(self, e): # 不理解画图
        """Add an element to the back of queue"""
        newest = self._Node(e, None) # node will be new tail node
        if self.is_empty():
            self._head = newest # special case:previously empty
        else:
            self._tail._pointer = newest
        self._tail = newest
        self._size += 1

    def printer_queue(self):
        node = self._head
        while node:
            print(node._element, end = " ")
            node = node._pointer
        print()


def quick_sort(S):
    """Sort the elements of queue S using the quick-sort algorithm"""
    n = len(S)
    if n < 2:
        return

    # 第一步
    # divide
    p = S.first() # using first as arbitrary pivot
    L = LinkedQueue()
    E = LinkedQueue()
    G = LinkedQueue()
    while not S.is_empty():
        if S.first() < p:
            L.enqueue(S.dequeue())
        elif p < S.first():
            G.enqueue(S.dequeue())
        else:
            E.enqueue(S.dequeue())

    # 第二步
    # conquer(with recursion)
    quick_sort(L) # sort elements less than p
    quick_sort(G) # sort elements grater than p

    # 第三步
    # concatenae results
    while not L.is_empty():
        S.enqueue(L.dequeue())
    while not E.is_empty():
        S.enqueue(E.dequeue())
    while not G.is_empty():
        S.enqueue(G.dequeue())

In [2]:
lq = LinkedQueue()
lq.enqueue(24)
lq.enqueue(45)
lq.enqueue(63)
lq.enqueue(85)
lq.enqueue(17)
lq.enqueue(31)
lq.enqueue(50)
lq.enqueue(96)
lq.printer_queue()

quick_sort(lq)
lq.printer_queue()

24 45 63 85 17 31 50 96 
17 24 31 45 50 63 85 96 


### 下面程序的运行过程

<img src="../../Other/img/快速排序.png">

In [3]:
def inplace_quick_sort(S, a, b): # 对Python列表S的就地快速排序(不需要额外的容器空间)
    """Sort the list from S[a] to S[b] inclusive using the quick-sort algorithm"""
    if a >= b: # range is trivially sorted
        return
    # 最后一个元素作为基准值
    pivot = S[b] # last element of range is pivot
    left = a # will scan rightward
    right = b - 1 # will scan leftward
    while left <= right:
        # scan until reaching value equal or larger the pivot(or right marker)
        while left <= right and S[left] < pivot:
            left += 1
        # scan until reaching value equal or smaller the pivot(or left marker)
        while left <= right and pivot < S[right]:
            right -= 1
        if left <= right:
            S[left], S[right] = S[right], S[left] # swap values
            # 交换一次后,原来的left必定小于pivot,原来的right必定大于privot,故此时left += 1,right -= 1
            left, right = left + 1, right - 1 # shrink range
    # 上图中最后left=4,right=3,left>right,跳出循环

    # put privot into its final place(currently marked by left index)
    S[left], S[b] = S[b], S[left] # 交换最后一个left位置处的元素和最后一个元素,得到最后一个元素正确的位置

    # 递归地解决剩下两边的子序列
    inplace_quick_sort(S, a, left - 1)
    inplace_quick_sort(S, left + 1, b)

In [4]:
lst = [85, 24, 63, 45, 17, 31, 96, 50]
print(lst)
inplace_quick_sort(lst, 0, 7)
print(lst)

[85, 24, 63, 45, 17, 31, 96, 50]
[17, 24, 31, 45, 50, 63, 85, 96]


基准值的选择:在实践中,一种选择基准值的常用技巧是取数组的头部,中部和尾部的值的中位数.这种三数取中的启发式搜索将更多地选择到好的基准值,对于更大的数据集合,可能需要计算多余三个潜在基准值的中值